<a href="https://colab.research.google.com/github/camstillo/electrodynamics/blob/master/Midterm_Project_PHY330.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    # Import gmsh library for generating meshes.
    import gmsh
except ImportError:
    # If it is not available, install it.  Then import it.
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
    import gmsh

--2022-10-30 23:02:16--  https://fem-on-colab.github.io/releases/gmsh-install.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2403 (2.3K) [application/x-sh]
Saving to: ‘/tmp/gmsh-install.sh’

/tmp/gmsh-install.s 100%[===================>]   2.35K  --.-KB/s    in 0s      

2022-10-30 23:02:16 (34.1 MB/s) - ‘/tmp/gmsh-install.sh’ saved [2403/2403]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ GMSH_INSTALLED=/usr/local/share/fem-on-colab/gmsh.installed
+ [[ ! -f /usr/local/share/fem-on-colab/gmsh.installed ]]
+ H5PY_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/d48f2be/releases/h5py-install.sh
+ [[ https://githu

In [2]:
try:
    # Import FEniCSx libraries for finite element analysis.
    import dolfinx
except ImportError:
    # If they are not found, install them.  Then import them.
    !wget "https://fem-on-colab.github.io/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

--2022-10-30 23:03:34--  https://fem-on-colab.github.io/releases/fenicsx-install-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3250 (3.2K) [application/x-sh]
Saving to: ‘/tmp/fenicsx-install.sh’

/tmp/fenicsx-instal 100%[===================>]   3.17K  --.-KB/s    in 0s      

2022-10-30 23:03:34 (49.1 MB/s) - ‘/tmp/fenicsx-install.sh’ saved [3250/3250]

+ INSTALL_PREFIX=/usr/local
++ awk -F/ '{print NF-1}'
++ echo /usr/local
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICSX_INSTALLED=/usr/local/share/fem-on-colab/fenicsx.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenicsx.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/af0f467/releases/pybind1

In [3]:
try:
    # Import multiphenicsx, mainly for plotting.
    import multiphenicsx
except ImportError:
    # If they are not found, install them.
    !pip3 install "multiphenicsx@git+https://github.com/multiphenics/multiphenicsx.git@8b97b4e"
    import multiphenicsx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/multiphenics/multiphenicsx.git (to revision 8b97b4e) to /tmp/pip-install-9apbbt53/multiphenicsx_ac77b884c8fe4e308fcb3370eb1d8cb9
  Running command git clone -q https://github.com/multiphenics/multiphenicsx.git /tmp/pip-install-9apbbt53/multiphenicsx_ac77b884c8fe4e308fcb3370eb1d8cb9
  Running command git checkout -q 8b97b4e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for multiphenicsx: filename=multiphenicsx-0.2.dev1-py3-none-any.whl size=42586 sha256=6c455f1a36a8c243cd422c512708333f8c2182ca57fbc019153d996a484fd366
  Stored in directory: /tmp/pip-ephem-wheel-cache-c8evdc2l/wheels/a4/44/a9/e578d5b67de6acd8e975bd56c96ee36944a693ee3d7bc91026
Successfully built multiphenicsx


In [4]:
# Everything should be installed now.
# Import the rest of what we need.

import dolfinx.fem
import dolfinx.io
import gmsh
import mpi4py.MPI
import numpy as np
import petsc4py.PETSc
import ufl
import multiphenicsx.fem
import multiphenicsx.io

In [46]:
# Define the center of the circle.
x0 = 0
y0 = 0
z0 = 0

#Define the top magnet
P1X = -2.5
P1Y = 4.33
P2X = -2.5
P2Y = 2.17
P3X = 2.5
P3Y = 2.17
P4X = 2.5
P4Y = 4.33

#Define Bottom Magnet
B1X = -2.5
B1Y = -4.33
B2X = -2.5
B2Y = -1.76
B3X = -2
B3Y = -1.76
B4X = -2
B4Y = -2.76
B5X = 2
B5Y = -2.76
B6X = 2
B6Y = -1.76
B7X = 2.5
B7Y = -1.76
B8X = 2.5
B8Y = -4.33

 
# Define the radius of the circle.
r0 = 5

# Tell the modeling program how many dimensions we are using.
dim = 2

# Grid size parameter.  Make it smaller for higher resolution.
delta = 0.2

In [47]:
# Create a model.
gmsh.initialize()
gmsh.model.add("mesh")

In [64]:
# Define points: center of circle and two points on opposite sides.
p0 = gmsh.model.geo.addPoint(x0,y0, z0, delta)
p1 = gmsh.model.geo.addPoint(x0, y0-r0, z0, delta)
p2 = gmsh.model.geo.addPoint(x0, y0+r0,z0, delta)

#Define points: Top Magnet
T1 = gmsh.model.geo.addPoint(P1X, P1Y, z0, delta)
T2 = gmsh.model.geo.addPoint(P2X, P2Y, z0, delta)
#T3 = gmsh.model.geo.addPoint(x0, y0, z0, delta)
T4 = gmsh.model.geo.addPoint(P3X, P3Y, z0, delta)
T5 = gmsh.model.geo.addPoint(P4X, P4Y, z0, delta)

#Define points: Bottom Magnet
B1 = gmsh.model.geo.addPoint(B1X, B1Y, z0, delta)
B2 = gmsh.model.geo.addPoint(B2X, B2Y, z0, delta)
B3 = gmsh.model.geo.addPoint(B3X, B3Y, z0, delta)
B4 = gmsh.model.geo.addPoint(B4X, B4Y, z0, delta)
B5 = gmsh.model.geo.addPoint(B5X, B5Y, z0, delta)
B6 = gmsh.model.geo.addPoint(B6X, B6Y, z0, delta)
B7 = gmsh.model.geo.addPoint(B7X, B7Y, z0, delta)
B8 = gmsh.model.geo.addPoint(B8X, B8Y, z0, delta)

#Define: Top magnet shape
L1 = gmsh.model.geo.addLine(T1, T2)
L2 = gmsh.model.geo.addLine(T2, p0)
L3 = gmsh.model.geo.addLine(p0, T4)
L4 = gmsh.model.geo.addLine(T4, T5)
topLoop = gmsh.model.geo.addCurveLoop([L1, L2, L3, L4])

#Define: Bottom magnet shape
D1 = gmsh.model.geo.addLine(B1, B2)
D2 = gmsh.model.geo.addLine(B2, B3)
D3 = gmsh.model.geo.addLine(B3, B4)
D4 = gmsh.model.geo.addLine(B4, B5)
D5 = gmsh.model.geo.addLine(B5, B6)
D6 = gmsh.model.geo.addLine(B6, B7)
D7 = gmsh.model.geo.addLine(B7, B8)
bottomLoop = gmsh.model.geo.addCurveLoop([D1, D2, D3, D4, D5, D6, D7])

# Define two semicircular arcs that will be joined into a circle.
c0 = gmsh.model.geo.addCircleArc(p1, p0, p2)
c1 = gmsh.model.geo.addCircleArc(p2, p0, p1)
loop = gmsh.model.geo.addCurveLoop([c0,c1])

## Alternate geometry: semicircle
# l0 = gmsh.model.geo.addLine(p2, p1)
# loop = gmsh.model.geo.addCurveLoop([c0, l0])

disk = gmsh.model.geo.addPlaneSurface([loop])
topMagnet = gmsh.model.geo.addPlaneSurface([topLoop])
bottomMagnet = gmsh.model.geo.addPlaneSurface([bottomLoop])

# Update the model with all of the features we add.
gmsh.model.geo.synchronize()

# Some geometric objects were only used to define others.
# Identify the physical objects.
gmsh.model.addPhysicalGroup(1, [c0,c1], 2)
gmsh.model.addPhysicalGroup(2, [disk], 3)
gmsh.model.addPhysicalGroup(2, [topMagnet], 4)
gmsh.model.addPhysicalGroup(2, [bottomMagnet], 5)

Exception: ignored

In [56]:
# Create a mesh for this system.
gmsh.model.mesh.generate(dim)

# Bring the mesh into FEniCSx.
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)

# Close the mesh generating program.
gmsh.finalize()

AssertionError: ignored

In [ ]:
# Plot the entire mesh.
multiphenicsx.io.plot_mesh(mesh)

In [ ]:
# Plot the subdomains that FEniCSx has identified.
# There should only be one for this model.
multiphenicsx.io.plot_mesh_tags(subdomains)

In [ ]:
# Inspect the boundaries of the elements and the system.
multiphenicsx.io.plot_mesh_tags(boundaries)